# Ejercicio OpenStreetMap
Para este ejercicio vas a trabar con Overpass, que es la API que utiliza los datos de OpenStreetMap. Tras obtener datos geográficos mediante la API, tendrás que representarlos con Plotly.

Hay que obtener los siguientes datos de cada ubicación conseguida en OpenStreetMap
* tipo de amenity
* latitud
* longitud
* nombre
* direccion

**NOTA**: En este Notebook utilizarás gráficas de Plotly, y JupyterLab de momento no lo soporta, a no ser que instales una extensión. Para solucionar esto, abre el Notebook desde Jupyter normal:

![imagen](./img/jupyter.png)

Tienes en [este enlace](https://wiki.openstreetmap.org/wiki/Key:amenity#Healthcare) la lista de *amenities* que vas a usar en el notebook.

## Instalación
Si no tienes instalado Plotly, abre un terminal y escribe:
```
pip install plotly
```

Importa las librerias que vamos a usar en el Notebook

In [174]:
import requests
import json
import pandas as pd

Ataca a la API de OpenStreetmap con el siguiente código de ejemplo

In [175]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="ice_cream"]
  (40.420801, -3.694702,40.423754, -3.688167); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()

<Response [200]>


Con el siguiente código podrás adaptar el JSON que devuelve la API, a un formato DataFrame más amigable

In [176]:
def json_to_df(data):
    elements = data['elements']
    places = {'tipo': [], 'lat': [], 'lon': [], 'name': [], 'address': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['tipo'].append(tipo)
        places['lat'].append(latitude)
        places['lon'].append(longitude)
        places['name'].append(name)
        places['address'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)

Utilizarás esta función para representar los datos en un mapa de Matplotlib

In [177]:
import plotly.express as px

mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"

def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()

Como ejemplo, si quiero ver cuántos restaurantes tengo cerca de The Bridge:

In [178]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="restaurant"]
  (40.420801, -3.694702,40.423754, -3.688167); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,restaurant,40.423086,-3.693547,La Castafiore,NO STREET 9999
1,restaurant,40.422607,-3.693256,Sake Dining Himawari,NO STREET 9999
2,restaurant,40.423315,-3.692506,Ainhoa,Calle de Doña Bárbara de Braganza 12
3,restaurant,40.421960,-3.694270,La Kitchen,NO STREET 9999
4,restaurant,40.423695,-3.691651,Café Espejo,NO STREET 9999
5,restaurant,40.421951,-3.693535,Tampu,Calle de Prim 13
6,restaurant,40.422266,-3.694097,Casa Gades,Calle del Conde de Xiquena 4
7,restaurant,40.423151,-3.694062,El Cano de Xiquena,NO STREET 9999
8,restaurant,40.423345,-3.692632,El Gato Persa,Calle de Doña Bárbara de Braganza 10
9,restaurant,40.422441,-3.694220,Cannibal Raw Bar,Calle del Almirante 12


In [179]:
pinta_mapa(df)

## Ejercicio 1
### Búsqueda de aparcamiento
Para este caso de uso imagina que vas a Madrid de viaje y estás buscando posibles parkings donde dejar el coche, crea un DataFrame con todos los parkings (de OpenStreetMap) de la ciudad de Madrid. Para ello define una cuadrícula con dos coordenadas sobre Madrid.

Representa en un mapa los resultados

In [180]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="parking"]
(40.4366, -3.7043, 41.3909, -3.6174); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()

<Response [200]>


In [181]:
def json_to_df(data):
    elements = data['elements']
    places = {'tipo': [], 'lat': [], 'lon': [], 'name': [], 'address': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['tipo'].append(tipo)
        places['lat'].append(latitude)
        places['lon'].append(longitude)
        places['name'].append(name)
        places['address'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)

In [182]:
import plotly.express as px

mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"

def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()

In [183]:
pinta_mapa(df)

## Mudanza
Imagina que te quieres mudar al Berrueco, pero antes te gustaría obtener algún indicador de si hay civilización por esa zona. Mira a ver si existe gasolineras y/o colegios por la zona. Define, al igual que en el caso anterior, un rectángulo mediante dos coordenadas. Busca la manera de realizar una única petición a la API.

Representa después en un mapa, por color los colegios y gasolineras

In [184]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"~"school|fuel"]
  (40.8637,-3.6071,40.9105,-3.5136);
out;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
print(response)
data = response.json()
df_muda = json_to_df(data)
df_muda

<Response [200]>


,tipo,lat,lon,name,address
0,music_school,40.872876,-3.606421,Escuela Municipal De Musica Y Danza La Cabrera,NO STREET 96
1,fuel,40.875833,-3.602389,Shell,Avenida de La Cabrera 9999


In [185]:
def json_to_df(data):
    elements = data['elements']
    places = {'tipo': [], 'lat': [], 'lon': [], 'name': [], 'address': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['tipo'].append(tipo)
        places['lat'].append(latitude)
        places['lon'].append(longitude)
        places['name'].append(name)
        places['address'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)

In [186]:
import plotly.express as px

mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"

def pinta_mapa(df_muda):
    fig = px.scatter_mapbox(df_muda,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()

In [187]:
pinta_mapa(df_muda)

## Zonas COVID
Imagina que trabajas en el ayuntamiento de tu barrio/pueblo/ciudad, y por motivos del COVID tienes que identificar:
1. Todas las zonas abiertas de uso común (zonas de barbacoas, fuentes de agua y parques infantiles (kindergarten)), para cerrarlas.
2. Todos los hospitales y clínicas de la zona.

Como no sabes muy bien qué puntos serán, utilizas la API de OpenStreetMap. Representa todo en un único mapa

In [188]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"~"bbq|drinking_water|kindergarten|clinic|hospital"]

  (37.3369,-6.0772,37.4322,-5.8736);
out;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
print(response)
data = response.json()
df_covid = json_to_df(data)
df_covid

<Response [200]>


,tipo,lat,lon,name,address
0,drinking_water,37.371685,-6.049114,NO NAME,NO STREET 9999
1,drinking_water,37.387834,-6.001198,Fuente para beber agua potable,NO STREET 9999
2,drinking_water,37.393873,-5.983184,NO NAME,NO STREET 9999
3,drinking_water,37.393116,-5.986985,NO NAME,NO STREET 9999
4,drinking_water,37.392572,-5.989869,NO NAME,NO STREET 9999
...,...,...,...,...,...
398,clinic,37.376331,-6.001857,Alfamedica Centro de Rehabilitación,Calle Pedro Pérez Fernández 7
399,clinic,37.388650,-6.011749,NO NAME,Calle Rafael Belmonte García 9999
400,drinking_water,37.385240,-5.998645,NO NAME,NO STREET 9999
401,drinking_water,37.347229,-5.980046,NO NAME,NO STREET 9999


In [189]:
def json_to_df(data):
    elements = data['elements']
    places = {'tipo': [], 'lat': [], 'lon': [], 'name': [], 'address': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['tipo'].append(tipo)
        places['lat'].append(latitude)
        places['lon'].append(longitude)
        places['name'].append(name)
        places['address'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)

In [190]:
import plotly.express as px

mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"

def pinta_mapa(df_muda):
    fig = px.scatter_mapbox(df_muda,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()

In [191]:
pinta_mapa(df_covid)

## Supermercados
Para este ejemplo supondrás que trabajas en una cadena de supermercados en la que se está planteando construir un nuevo local en la provincia de Sevilla. Busca mediante la API de Overpass todos los supermercados de la provincia para poder comprobar en qué nucleos urbanos habría poca competencia.

En este caso hay que buscar por área, no por cuadrícula! Investiga en la documentación/Stackoverflow cómo se haría eso.

In [192]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["shop"="supermarket"]
  (37.1658,-6.3556,37.5473,-5.5412);
out;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
print(response)
data = response.json()
df_super = json_to_df(data)
df_super

<Response [200]>


,tipo,lat,lon,name,address
0,None,37.373002,-6.047204,MAS,NO STREET 9999
1,None,37.390016,-5.973769,Maxi Dia,NO STREET 9999
2,None,37.406364,-5.976342,El Jamón,NO STREET 9999
3,None,37.406508,-5.973342,Valdejimena,Avenida de San Juan de la Salle 9999
4,None,37.406889,-5.972455,Dia,Avenida de San Juan de la Salle 12
...,...,...,...,...,...
292,None,37.376837,-5.974045,Dia,Avenida Ramón y Cajal 12
293,None,37.511615,-6.025710,NO NAME,NO STREET 9999
294,None,37.513835,-6.024880,Tienda Pepa,NO STREET 9999
295,None,37.380747,-5.966079,CashFresh,Avenida de la Ciudad Jardin 4


In [193]:
def json_to_df(data):
    elements = data['elements']
    places = {'tipo': [], 'lat': [], 'lon': [], 'name': [], 'address': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['tipo'].append(tipo)
        places['lat'].append(latitude)
        places['lon'].append(longitude)
        places['name'].append(name)
        places['address'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)

In [194]:
import plotly.express as px

mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"

def pinta_mapa(df_super):
    fig = px.scatter_mapbox(df_super,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()

In [195]:
pinta_mapa(df_super)

## Etiqueta en el mapa
Busca la manera de modificar la función `pinta_mapa()`, de tal manera que cuando pases el ratón por encima de cada punto, aparezca el nombre de cada nodo. Por ejemplo, que en el último apartado aparezcan los nombres de los supermercados.

In [196]:
df_super2 = df_super.copy()
df_super2["tipo"] = df_super["name"]
df_super2

,tipo,lat,lon,name,address
0,MAS,37.373002,-6.047204,MAS,NO STREET 9999
1,Maxi Dia,37.390016,-5.973769,Maxi Dia,NO STREET 9999
2,El Jamón,37.406364,-5.976342,El Jamón,NO STREET 9999
3,Valdejimena,37.406508,-5.973342,Valdejimena,Avenida de San Juan de la Salle 9999
4,Dia,37.406889,-5.972455,Dia,Avenida de San Juan de la Salle 12
...,...,...,...,...,...
292,Dia,37.376837,-5.974045,Dia,Avenida Ramón y Cajal 12
293,NO NAME,37.511615,-6.025710,NO NAME,NO STREET 9999
294,Tienda Pepa,37.513835,-6.024880,Tienda Pepa,NO STREET 9999
295,CashFresh,37.380747,-5.966079,CashFresh,Avenida de la Ciudad Jardin 4


In [197]:
pinta_mapa(df_super2)